In [15]:
import telegram, logging, soundfile, aifc, subprocess, os

from telegram.ext import Updater, CommandHandler, Filters, MessageHandler

from chatbotwrapper import PythonChatBot
from io import BytesIO

In [2]:
chatbot = PythonChatBot(threshold=0.5)

In [3]:
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)

In [4]:
bot_train = telegram.Bot(token='386950793:AAFWQO5Exopgk-WwzHcrKEArN7tVM20OL4A')

updater_train = Updater(token='386950793:AAFWQO5Exopgk-WwzHcrKEArN7tVM20OL4A')

dispatcher_train = updater_train.dispatcher

In [5]:
def start_train(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text="""Привет! Я готов обучаться!\nПришли сообщением вопрос и ответ на него.\nНе забудь разделить их вот этим симоволом "|" """)

In [6]:
def train(bot, update):
    user = update.message.from_user
    text = update.message.text
    logging.info("text of %s: %s" % (user.first_name, text))
    bot.send_message(chat_id=update.message.chat_id, text="""Спасибо, я постараюсь хорошо обучиться.\nПокидай обучалок еще, мне нравится учиться""")
    try:
        train_text = text.split('|')
        chatbot.train(train_text[0], train_text[1])
    except Exception as e:
        logging.warning("Произошла ошибка -{}- \nНа обучающем тексте -{}-".format(e, text))
        bot.send_message(chat_id=update.message.chat_id, text="Не получилось обучиться. Может быть неправильный формат?")
    else:
        logging.info("Обучение на обучающем тексте -{}- прошло успешно".format(text))

In [7]:
start_handler_train = CommandHandler('start', start_train)
text_handler_train = MessageHandler(Filters.text, train)

dispatcher_train.add_handler(start_handler_train)
dispatcher_train.add_handler(text_handler_train)

In [8]:
updater_train.start_polling()

In [9]:
bot_predict = telegram.Bot(token='383933408:AAHmUeE2FX_tODZfiqZUSDMNHYuO-31q1p4')

updater_predict = Updater(token='383933408:AAHmUeE2FX_tODZfiqZUSDMNHYuO-31q1p4')

dispatcher_predict = updater_predict.dispatcher

chatbot_predict = PythonChatBot(threshold=0.5, read_only=True)

2017-09-16 21:53:10,791 - pymorphy2.opencorpora_dict.wrapper - INFO - Loading dictionaries from /Users/max/anaconda/lib/python3.6/site-packages/pymorphy2_dicts/data
2017-09-16 21:53:10,867 - pymorphy2.opencorpora_dict.wrapper - INFO - format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


In [10]:
def start_predict(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text="""Привет! Меня еще учат, но кажется что я более-менее готов отвечать на вопросы""")

In [11]:
def predict(bot, update):
    user = update.message.from_user
    text = update.message.text
    logging.info("text of %s: %s" % (user.first_name, text))
    try:
        response = chatbot_predict.response(text)
    except Exception as e:
        bot.send_message(chat_id=update.message.chat_id, text="К сожалению что-то пошло не так")
        logging.warning("Произошла ошибка -{}- \nПытался ответить на -{}-".format(e, text))
    else:
        bot.send_message(chat_id=update.message.chat_id, text=response)
        logging.info("Ответа на вопрос -{}- прошел успешно".format(text))

In [12]:
def predict_sound(bot, update):
    import subprocess
    user = update.message.from_user
    filepath = '{}_question'.format(user.first_name)
    #filepath = 'woman1_wb.wav'
    try:
        file_id = update.message.voice.file_id
        newFile = bot.get_file(file_id)
        newFile.download(filepath + '_wb.mp3')
        subprocess.call(['ffmpeg', '-i', filepath + '_wb.mp3', filepath + '_wb.wav'])
        answer = chatbot_predict.response_sound(filepath + '_wb.wav')
        os.remove(filepath+'_wb.mp3')
        os.remove(filepath+'_wb.wav')
        bot.send_message(chat_id=update.message.chat_id, text=str(answer))
    except Exception as e:
        logging.warning("Произошла ошибка -{}- \nПытался ответить на звук".format(e))
        bot.send_message(chat_id=update.message.chat_id, text="К сожалению что-то пошло не так")

In [13]:
start_handler_predict = CommandHandler('start', start_predict)
text_handler_predict = MessageHandler(Filters.text & (~Filters.voice), predict)
voice_handler_predict = MessageHandler(Filters.voice, predict_sound)

dispatcher_predict.add_handler(start_handler_predict)
dispatcher_predict.add_handler(text_handler_predict)
dispatcher_predict.add_handler(voice_handler_predict)

In [14]:
updater_predict.start_polling()

2017-09-16 21:53:20,279 - chatterbot.adapters - INFO - Recieved input statement: 12345678
2017-09-16 21:53:20,294 - chatterbot.adapters - INFO - "12345678" is not a known statement
2017-09-16 21:53:20,328 - chatterbot.adapters - INFO - Using "12345678" as a close match to "12345678"
2017-09-16 21:53:20,343 - chatterbot.adapters - INFO - No response to "12345678" found. Selecting a random response.
2017-09-16 21:53:20,345 - chatterbot.adapters - INFO - BestMatch selected "тыща пыхать тырыпыхать" as a response with a confidence of 0
2017-09-16 21:53:20,377 - chatterbot.adapters - INFO - LowConfidenceAdapter selected "Я недоучился, поэтому не могу понять о чем речь" as a response with a confidence of 1
2017-09-16 21:53:20,391 - chatterbot.adapters - INFO - NoKnowledgeAdapter selected "12345678" as a response with a confidence of 0
2017-09-16 21:53:33,180 - chatterbot.adapters - INFO - Recieved input statement: 12345678
2017-09-16 21:53:33,186 - chatterbot.adapters - INFO - "12345678" is n